In [108]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [96]:
listings = pd.read_csv("data/listings.csv.gz")
reviews = pd.read_csv("data/reviews.csv.gz")

In [102]:
listings.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20201223070513,2020-12-23,Ocean front condo on the sand,"Missionbeachretreat. Great Ocean Front Condo, right on the boardwalk. We have many reviews on website ""V"".<br /><br /><b>The space</b><br />This charming two bedroom beachfront condominium offers stunning ocean and sunset views from the living room, master bedroom, as well as the spacious outdoor balcony. This is the perfect setting for a romantic getaway or a family vacation. Strolls on the white sandy beach directly in front as well as limitless water sports are all just out your door.<br /><br />This middle floor corner unit condo features a sun filled living room with a flat screen TV and comfortable seating for up to six guests. Guests will also enjoy the fully equipped kitchen and dining room with seating for 4. Guests staying in the finely appointed master bedroom will enjoy waking up every morning to sprawling and unobstructed white water views through the oversized bedroom windows. There is also a second guest bedroom with two twin beds and an inflatable queen size, aer","While some may compare San Diego’s Mission Beach to the East Coast’s Atlantic City and Coney Island, there is a unique Southern California vibe you can’t find anywhere else here. Nestled between Pacific Beach on the north side, and Belmont Park on its southern side, Mission Beach offers a lively and robust variety of activities in the sun, surf and sand. From hot dog stands to fine dining eateries, hot night clubs to street performers, expect to find all kinds of free-spirited beach goers in a fun, carnival-like atmosphere along the sandy shores.",https://a0.muscache.com/pictures/d3342744-140a-445e-a487-18f33409005c.jpg,8435,https://www.airbnb.com/users/show/8435,Jef Karchin,2009-02-23,"San Diego, California, United States","Real estate broker, Jef Karchin real estate I provide flat fee real estate services to For Sale By Owners. I stay at the condo whenever it is not rented. There is nothing like staying at the beach, right on the ocean. I surf right in front of our condo and always enjoy watching the dolphins swimming by, occasionally joining them in the waves.",within a few hours,100%,100%,f,https://a0.muscache.com/im/users/8435/profile_pic/1391638355/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/8435/profile_pic/1391638355/original.jpg?aki_policy=profile_x_medium,Mission Beach,3.0,3.0,"['email', 'phone', 'kba']",t,t,"San Diego, California, United States",Mission Bay,NaN,32.78430,-117.25258,Entire condominium,Entire home/apt,6,NaN,1 bath,2.0,4.0,"[""Iron"", ""Elevator"", ""Stove"", ""Microwave"", ""Dishes and silverware"", ""Beachfr

In [101]:
long_content_cols = ["description", "neighborhood_overview", "picture_url", 
                     "host_url", "host_about", "host_thumbnail_url", "host_picture_url"]
listings.drop(long_content_cols, axis=1).head()

,id,listing_url,scrape_id,last_scraped,name,host_id,host_name,host_since,host_location,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,property_type,room_type,accommodates,bathrooms,bathrooms_text,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,minimum_minimum_nights,maximum_minimum_nights,minimum_maximum_nights,maximum_maximum_nights,minimum_nights_avg_ntm,maximum_nights_avg_ntm,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,number_of_reviews_ltm,number_of_reviews_l30d,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5570,https://www.airbnb.com/rooms/5570,20201223070513,2020-12-23,Ocean front condo on the sand,8435,Jef Karchin,2009-02-23,"San Diego, California, United States",within a few hours,100%,100%,f,https://a0.muscache.com/im/users/8435/profile_pic/1391638355/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/8435/profile_pic/1391638355/original.jpg?aki_policy=profile_x_medium,Mission Beach,3.0,3.0,"['email', 'phone', 'kba']",t,t,"San Diego, California, United States",Mission Bay,NaN,32.78430,-117.25258,Entire condominium,Entire home/apt,6,NaN,1 bath,2.0,4.0,"[""Iron"", ""Elevator"", ""Stove"", ""Microwave"", ""Dishes and silverware"", ""Beachfront"", ""Heating"", ""Kitchen"", ""Washer"", ""Refrigerator"", ""Smoke alarm"", ""Hair dryer"", ""Cable TV"", ""Dryer"", ""Dishwasher"", ""Patio or balcony"", ""BBQ grill"", ""TV"", ""Oven"", ""Coffee maker"", ""Long term stays allowed"", ""Essentials"", ""Dedicated workspace"", ""Wifi"", ""Waterfront"", ""Free parking on premises"", ""Cooking basics""]","$2,050.00",3,365,3.0,3.0,365.0,365.0,3.0,365.0,NaN,t,6,21,44,207,2020-12-23,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,3,3,0,0,NaN
1,29967,https://www.airbnb.com/rooms/29967,20201223070513,2020-12-23,"Great home, 10 min walk to Beach",129123,Michael,2010-05-21,"San Diego, California, United States",within an hour,100%,50%,t,https://a0.muscache.com/im/users/129123/profile_pic/1391751162/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/129123/profile_pic/1391751162/original.jpg?aki_policy=profile_x_medium,Pacific Beach,6.0,6.0,"['email', 'phone', 'reviews', 'kba', 'work_email']",t,t,NaN,Pacific Beach,NaN,32.80724,-117.25630,Entire bungalow,Entire home/apt,8,NaN,2 baths,2.0,3.0,"[""Iron"", ""Hot water"", ""Heating"", ""Kitchen"", ""Washer"", ""First aid kit"", ""Pack \u2019n Play/travel crib"", ""Smoke alarm"", ""Shampoo"", ""Hair dryer"", ""Cable TV"", ""Dryer"", ""TV"", ""Long term stays allowed"", ""Fire extinguisher"", ""Essentials"", ""Dedicated workspace"", ""Wifi"", ""Free parking on premises"", ""Hangers"", ""Carbon monoxide alarm""]",$261.00,4,365,4.0,4.0,365.0,365.0,4.0,365.0,NaN,t,6,6,6,213,2020-12-23,62,2,1,2010-07-09,2020-11-29,98.0,10.0,10.0,10.0,10.0,10.0,10.0,NaN,f,5,5,0,0,0.49
2,38245,https://www.airbnb.com/rooms/38245,20201223070513,2020-12-23,Point Loma: Den downstairs,164137,Melinda,2010-07-12,"San Diego, California, United States",within a day,100%,86%,f,https://a0.muscache.com/im/users/164137/profile_pic/1308609474/original.jpg?aki_policy=profile_small,https://a0.muscache.com/im/users/164137/profile_pic/1308609474/original.jpg?aki_policy=profile_x_medium,Loma Portal,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"San Diego, California, United 

In [89]:
df_numeric = listings.select_dtypes(exclude=['object'])
useful_cat_varibles = ["neighbourhood_cleansed", "host_identity_verified", "host_neighbourhood"]
useful_cat_varibles += ["property_type", "room_type"]
useful_cat_varibles += ["host_response_time"]
df_cat = listings[useful_cat_varibles]


description_cols = ["description", "host_about"]
s_amenities = listings.amenities


In [117]:
listings.neighbourhood_cleansed.value_counts()

Mission Bay                    1453
Pacific Beach                  727 
La Jolla                       655 
East Village                   546 
Ocean Beach                    431 
Midtown                        424 
North Hills                    420 
Loma Portal                    227 
Little Italy                   205 
Balboa Park                    200 
Gaslamp Quarter                178 
West University Heights        177 
Old Town                       175 
Mira Mesa                      162 
University City                147 
Grant Hill                     140 
Bay Park                       137 
Park West                      134 
Carmel Valley                  126 
Core                           121 
Normal Heights                 121 
Clairemont Mesa                116 
Cortez Hill                    114 
Serra Mesa                     110 
Marina                         106 
City Heights West              79  
North Clairemont               78  
Northwest                   

In [120]:
listings.price = listings.price.str.replace("$","").str.replace(",","").astype(float)

In [126]:
listings.groupby(["neighbourhood_cleansed"])["price"].agg(["count", "mean"]).sort_values(by=["count"])

,count,mean
neighbourhood_cleansed,,
Tijuana River Valley,1,360.000000
Lincoln Park,2,63.500000
Amphitheater And Water Park,2,62.000000
Palm City,2,102.500000
Eastlake Woods,2,127.500000
Horton Plaza,3,257.000000
Eastlake Trails,3,130.666667
East Lake,4,123.750000
Paseo Ranchoero,4,98.500000


In [109]:
listings.isnull().sum()

id                                              0   
listing_url                                     0   
scrape_id                                       0   
last_scraped                                    0   
name                                            0   
description                                     176 
neighborhood_overview                           2626
picture_url                                     0   
host_id                                         0   
host_url                                        0   
host_name                                       3   
host_since                                      3   
host_location                                   16  
host_about                                      2775
host_response_time                              990 
host_response_rate                              990 
host_acceptance_rate                            630 
host_is_superhost                               3   
host_thumbnail_url                            

In [106]:
listings["host_has_profile_pic"].value_counts()

t    9225
f    21  
Name: host_has_profile_pic, dtype: int64

In [30]:
useful_cat_varibles = ["neighbourhood_cleansed", "host_identity_verified", "host_neighbourhood"]
useful_cat_varibles += ["property_type", "room_type"]


"amenities" # length, some td-idf analysis

df_numeric = listings.select_dtypes(exclude=['object'])


## get rid of variable with only nan
exclude_list = [] 
for col in df.columns:
    value_cnts = len(df[col].unique())
    if value_cnts < 2:
        exclude_list.append(col)
        print(col)
        
        
exclude_list += ["scrape_id", "host_id"]
df = df.drop(exclude_list ,axis=1)


Y = listings.price.str.replace("$","").str.replace(",","").astype(float)


neighbourhood_group_cleansed
bathrooms
calendar_updated
license


,id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,minimum_nights_x,maximum_nights_x,...,listing_id,available,price,minimum_nights_y,maximum_nights_y,year,month,day,dayofyear,weekday
0,5570,3.0,3.0,32.784300,-117.252580,6,2.0,4.0,3,365,...,5570,f,2050.0,3.0,365.0,2020,12,23,358,2
1,5570,3.0,3.0,32.784300,-117.252580,6,2.0,4.0,3,365,...,5570,f,2050.0,3.0,365.0,2020,12,24,359,3
2,5570,3.0,3.0,32.784300,-117.252580,6,2.0,4.0,3,365,...,5570,f,2050.0,3.0,365.0,2020,12,25,360,4
3,5570,3.0,3.0,32.784300,-117.252580,6,2.0,4.0,3,365,...,5570,f,2050.0,3.0,365.0,2020,12,26,361,5
4,5570,3.0,3.0,32.784300,-117.252580,6,2.0,4.0,3,365,...,5570,f,2050.0,3.0,365.0,2020,12,27,362,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3375952,47202010,86.0,86.0,32.786237,-117.253391,8,3.0,5.0,3,1125,...,47202010,t,415.0,3.0,1125.0,2021,12,19,353,6
3375953,47202010,86.0,86.0,32.786237,-117.253391,8,3.0,5.0,3,1125,...,47202010,t,415.0,3.0,1125.0,2021,12,20,354,0
3375954,47202010,86.0,86.0,32.786237,-117.253391,8,3.0,5.0,3,1125,...,47202010,t,415.0,3.0,1125.0,2021,12,21,355,1
3375955,47202010,86.0,86.0,32.786237,-117.253391,8,3.0,5.0,3,1125,...,47202010,t,415.0,3.0,1125.0,2021,12,22,356,2


In [10]:
# encode category variable into numeric one
le = preprocessing.LabelEncoder()
categorical_feature = []
for col in df_all.columns:
    print(col, df_all[col].dtype)
    if df_all[col].dtype == 'object':
        le.fit(df_all[col])
        encoded_fts = le.transform(df_all[col])
        df_all[col] = encoded_fts
        categorical_feature.append(col)

id int64
host_listings_count float64
host_total_listings_count float64
latitude float64
longitude float64
accommodates int64
bedrooms float64
beds float64
minimum_nights_x int64
maximum_nights_x int64
minimum_minimum_nights float64
maximum_minimum_nights float64
minimum_maximum_nights float64
maximum_maximum_nights float64
minimum_nights_avg_ntm float64
maximum_nights_avg_ntm float64
availability_30 int64
availability_60 int64
availability_90 int64
availability_365 int64
number_of_reviews int64
number_of_reviews_ltm int64
number_of_reviews_l30d int64
review_scores_rating float64
review_scores_accuracy float64
review_scores_cleanliness float64
review_scores_checkin float64
review_scores_communication float64
review_scores_location float64
review_scores_value float64
calculated_host_listings_count int64
calculated_host_listings_count_entire_homes int64
calculated_host_listings_count_private_rooms int64
calculated_host_listings_count_shared_rooms int64
reviews_per_month float64
listing_id

In [11]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3375957 entries, 0 to 3375956
Data columns (total 46 columns):
 #   Column                                        Dtype  
---  ------                                        -----  
 0   id                                            int64  
 1   host_listings_count                           float64
 2   host_total_listings_count                     float64
 3   latitude                                      float64
 4   longitude                                     float64
 5   accommodates                                  int64  
 6   bedrooms                                      float64
 7   beds                                          float64
 8   minimum_nights_x                              int64  
 9   maximum_nights_x                              int64  
 10  minimum_minimum_nights                        float64
 11  maximum_minimum_nights                        float64
 12  minimum_maximum_nights                        float64
 1

In [129]:
df_all.head()

,id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,minimum_nights_x,maximum_nights_x,...,date,available,price,minimum_nights_y,maximum_nights_y,year,month,day,dayofyear,weekday
0,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-23,f,2050.0,3.0,365.0,2020,12,23,358,2
1,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-24,f,2050.0,3.0,365.0,2020,12,24,359,3
2,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-25,f,2050.0,3.0,365.0,2020,12,25,360,4
3,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-26,f,2050.0,3.0,365.0,2020,12,26,361,5
4,5570,3.0,3.0,32.7843,-117.25258,6,2.0,4.0,3,365,...,2020-12-27,f,2050.0,3.0,365.0,2020,12,27,362,6


In [96]:
df_all.describe()


,id,scrape_id,host_id,host_listings_count,host_total_listings_count,latitude,longitude,accommodates,bedrooms,beds,...,review_scores_location,review_scores_value,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,listing_id,minimum_nights_y,maximum_nights_y
count,3.375957e+06,3.375957e+06,3.375957e+06,3.374862e+06,3.374862e+06,3.375957e+06,3.375957e+06,3.375957e+06,3.012374e+06,3.358068e+06,...,2.818902e+06,2.818902e+06,3.375957e+06,3.375957e+06,3.375957e+06,3.375957e+06,2.850296e+06,3.375957e+06,3.375516e+06,3.375516e+06
mean,2.802539e+07,2.020122e+13,9.746568e+07,1.018693e+02,1.018693e+02,3.276951e+01,-1.171858e+02,4.724502e+00,1.898940e+00,2.510298e+00,...,9.809123e+00,9.444443e+00,1.784174e+01,1.574868e+01,2.035967e+00,4.108583e-02,1.908871e+00,2.802539e+07,6.692574e+00,8.532855e+05
std,1.404805e+07,0.000000e+00,9.910229e+07,5.071818e+02,5.071818e+02,6.469545e-02,6.426890e-02,3.158310e+00,1.215191e+00,1.996028e+00,...,5.998467e-01,8.470590e-01,3.590654e+01,3.451992e+01,1.262832e+01,4.141488e-01,1.904058e+00,1.404805e+07,1.682099e+01,4.277853e+07
min,5.570000e+03,2.020122e+13,8.435000e+03,0.000000e+00,0.000000e+00,3.254107e+01,-1.172814e+02,0.000000e+00,1.000000e+00,0.000000e+00,...,2.000000e+00,2.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e-02,5.570000e+03,1.000000e+00,1.000000e+00
25%,1.697281e+07,2.020122e+13,1.691311e+07,1.000000e+00,1.000000e+00,3.272700e+01,-1.172502e+02,2.000000e+00,1.000000e+00,1.000000e+00,...,1.000000e+01,9.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,4.200000e-01,1.697281e+07,2.000000e+00,3.620000e+02
50%,2.933944e+07,2.020122e+13,6.200869e+07,3.000000e+00,3.000000e+00,3.276092e+01,-1.171752e+02,4.000000e+00,1.000000e+00,2.000000e+00,...,1.000000e+01,1.000000e+01,3.000000e+00,2.000000e+00,0.000000e+00,0.000000e+00,1.270000e+00,2.933944e+07,2.000000e+00,1.125000e+03
75%,4.148234e+07,2.020122e+13,1.466791e+08,1.700000e+01,1.700000e+01,3.279865e+01,-1.171450e+02,6.000000e+00,2.000000e+00,3.000000e+00,...,1.000000e+01,1.000000e+01,1.400000e+01,1.100000e+01,1.000000e+00,0.000000e+00,2.840000e+00,4.148234e+07,4.000000e+00,1.125000e+03
max,4.720201e+07,2.020122e+13,3.809385e+08,4.355000e+03,4.355000e+03,3.308582e+01,-1.169357e+02,1.600000e+01,2.200000e+01,2.500000e+01,...,1.000000e+01,1.000000e+01,1.690000e+02,1.690000e+02,1.140000e+02,8.000000e+00,2.476000e+01,4.720201e+07,3.650000e+02,2.147484e+09


In [51]:
reviews.date = pd.to_datetime(reviews.date, format='%Y-%m-%d')

In [53]:
reviews.date.max(),  reviews.date.min()

(Timestamp('2020-12-23 00:00:00'), Timestamp('2010-07-09 00:00:00'))

In [66]:
calendar.head()

,listing_id,date,available,price,minimum_nights,maximum_nights
0,522108,2020-12-23,f,200.0,2.0,30.0
1,5570,2020-12-23,f,2050.0,3.0,365.0
2,5570,2020-12-24,f,2050.0,3.0,365.0
3,5570,2020-12-25,f,2050.0,3.0,365.0
4,5570,2020-12-26,f,2050.0,3.0,365.0


In [ ]:
property_type	room_type

In [19]:
host_response_rate, host_acceptance_rate

In [ ]:
property_type


In [70]:
listings.has_availability.value_counts()

t    9197
f      52
Name: has_availability, dtype: int64

In [58]:
(listings.host_neighbourhood == listings.neighbourhood_cleansed).value_counts()

False    6311
True     2938
dtype: int64

In [76]:
reviews.

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,29967,62788,2010-07-09,151260,Debbie,When I booked our stay in San Diego at Dennis ...
1,29967,64568,2010-07-14,141552,Eric,This was my first experience with using airbnb...
2,29967,67502,2010-07-22,141591,David,We found the house to be very accommodating--e...
3,29967,70466,2010-07-29,125982,Anders,As advertised and more. Dennis was very helpfu...
4,29967,74876,2010-08-07,29835,Miyoko,We had a great time in San Diego. Denis' house...
...,...,...,...,...,...,...
443411,46995597,715948454,2020-12-16,346790349,Mike,"This is a great place with modern design, extr..."
443412,46995597,716097352,2020-12-17,370055591,Hanna,Such a amazing place! It makes me feel like ho...
443413,46995597,716330508,2020-12-18,299951196,Trina,Booked this place last minute and was able to ...
443414,46995597,716579344,2020-12-19,74213915,Colby,We had a great time staying at this house. It ...


In [79]:
reviews.listing_id.unique().shape

(7808,)

In [77]:
reviews.reviewer_id.value_counts() # heavy reviewer

64785633     69
85972988     63
52458269     35
577119       30
88498882     28
             ..
53640388      1
102236152     1
41641543      1
334477245     1
219080271     1
Name: reviewer_id, Length: 397984, dtype: int64

In [75]:
reviews[reviews.listing_id == 29967].comments

0     When I booked our stay in San Diego at Dennis ...
1     This was my first experience with using airbnb...
2     We found the house to be very accommodating--e...
3     As advertised and more. Dennis was very helpfu...
4     We had a great time in San Diego. Denis' house...
                            ...                        
57    Michael’s place is great, very good location c...
58    Michaels home is spacious and clean. Was the p...
59    A very good neighborhood and house. The landlo...
60    Michael/Dennies were great hosts! Responded ve...
61          Great Location and nice for a small family.
Name: comments, Length: 62, dtype: object

In [ ]:
review_counts
review_senmentic_counts
